In [1]:
%reload_kedro

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2021-02-06 10:26:40,480 - root - INFO - ** Kedro project coursing-ml
2021-02-06 10:26:40,481 - root - INFO - Defined global variable `context` and `catalog`


In [2]:
df = catalog.load("preprocessed_udacity")
df.head(30)

2021-02-06 10:26:44,213 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_udacity` (CSVDataSet)...


,collaboration,description,difficulty,duration,free,id_course,n_reviews,rating,school,skills,title,url,time_range
0,Insight,"Learn to design data models, build data wareho...",intermediate,200.0,0,nd027,918,91.089325,School of Data Science,"Data Modeling, Data Pipelines, Data Lakes, Spa...",Become a Data Engineer,https://www.udacity.com/course/data-engineer-n...,months
1,"Google, Hootsuite, HubSpot, MailChimp, Moz, Fa...","Learn to create marketing content, use social ...",beginner,120.0,0,nd018,1753,93.964632,School of Business,"Digital Marketing, Social Media Marketing, Fac...",Become a Digital Marketer,https://www.udacity.com/course/digital-marketi...,months
2,NaN,You’ll learn how to create a digital user expe...,beginner,120.0,0,nd578,619,94.248788,School of Business,"User Research, User Experience, Prototyping, I...",Become a UX Designer,https://www.udacity.com/course/ux-designer-nan...,months
3,"AWS, Facebook Artificial Intelligence","Become an expert in neural networks, and learn...",intermediate,160.0,0,nd101,2893,93.218113,School of Artificial Intelligence,"Deep Learning, Neural Networks, Jupyter Notebo...",Deep Learning,https://www.udacity.com/course/deep-learning-n...,months
4,NaN,The goal of the Full Stack Web Developer Nanod...,intermediate,160.0,0,nd0044,517,90.870406,School of Programming & Development,"Python, JavaScript, PostgreSQL, Flask, Docker,...",Full Stack Web Developer,https://www.udacity.com/course/full-stack-web-...,months
5,NaN,"Learn the basics of programming through HTML, ...",beginner,160.0,0,nd000,3045,93.642036,School of Programming & Development,"Introduction to Programming, HTML, CSS, Python",Learn to Code,https://www.udacity.com/course/intro-to-progra...,months
6,NaN,Get hands-on experience by coding five real-wo...,intermediate,160.0,0,nd213,680,90.794118,School of Autonomous Systems,"Data Structures & Algorithms, Memory Managemen...",Become a C++ Developer,https://www.udacity.com/course/c-plus-plus-nan...,months
7,"Bertelsmann, Figure Eight, IBM Watson, Insight...",You’ll master the skills necessary to become a...,advanced,80.0,0,nd025,739,93.423545,School of Data Science,"Machine Learning, Deep Learning, Software Engi...",Become a Data Scientist,https://www.udacity.com/course/data-scientist-...,months
8,"Tableau, Mode","In this program, you’ll learn foundational dat...",beginner,120.0,0,nd098,1161,95.021533,School of Business,"Excel & Spreadsheets, SQL, Data Visualization,...",Business Analytics,https://www.udacity.com/course/business-analyt...,months
9,"Kaggle, AWS",Learn advanced machine learning techniques and...,intermediate,120.0,0,nd009t,761,91.537451,School of Artificial Intelligence,"Machine Learning, Supervised Learning, Unsuper...",Become a Machine Learning Engineer,https://www.udacity.com/course/machine-learnin...,months


In [3]:

corpus = df['title'].apply(lambda _: str(_))
corpus

c:\users\enrmeao\pycharmprojects\coursing\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0                      Become a Data Engineer
1                   Become a Digital Marketer
2                        Become a UX Designer
3                               Deep Learning
4                    Full Stack Web Developer
                        ...                  
245    Intro to Machine Learning with PyTorch
246              Monetization Strategy Course
247       Artificial Intelligence for Trading
248                AI Programming with Python
249                                 Learn SQL
Name: title, Length: 250, dtype: object

In [12]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('paraphrase-distilroberta-base-v1')

corpus = df['title'].apply(lambda _: str(_))

print(type(corpus))

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

queries = ['Learn to program', 'Evaluation of business']

2021-02-06 10:39:12,264 - root - INFO - Load pretrained SentenceTransformer: paraphrase-distilroberta-base-v1
2021-02-06 10:39:12,266 - root - INFO - Did not find folder paraphrase-distilroberta-base-v1
2021-02-06 10:39:12,268 - root - INFO - Try to download model from server: https://sbert.net/models/paraphrase-distilroberta-base-v1.zip
2021-02-06 10:39:12,270 - root - INFO - Load SentenceTransformer from folder: C:\Users\enrmeao/.cache\torch\sentence_transformers\sbert.net_models_paraphrase-distilroberta-base-v1
2021-02-06 10:39:14,484 - root - INFO - Use pytorch device: cpu
<class 'pandas.core.series.Series'>


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [23]:
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Corpus with example sentences
corpus = df['title'].apply(lambda _: str(_)).to_list()

print(type(corpus))
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['How to program', "Preparing for job interviews", "Make a web application from scratch"]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(idx)
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """

2021-02-06 10:50:33,445 - root - INFO - Load pretrained SentenceTransformer: paraphrase-distilroberta-base-v1
2021-02-06 10:50:33,449 - root - INFO - Did not find folder paraphrase-distilroberta-base-v1
2021-02-06 10:50:33,451 - root - INFO - Try to download model from server: https://sbert.net/models/paraphrase-distilroberta-base-v1.zip
2021-02-06 10:50:33,453 - root - INFO - Load SentenceTransformer from folder: C:\Users\enrmeao/.cache\torch\sentence_transformers\sbert.net_models_paraphrase-distilroberta-base-v1
2021-02-06 10:50:35,734 - root - INFO - Use pytorch device: cpu
<class 'list'>


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]





Query: How to program

Top 5 most similar sentences in corpus:
tensor(5)
Learn to Code (Score: 0.4848)
tensor(183)
What is Programming? (Score: 0.4447)
tensor(123)
Design of Computer Programs (Score: 0.4363)
tensor(154)
Introduction to Graduate Algorithms (Score: 0.3903)
tensor(218)
Programming for Data Science with R (Score: 0.3628)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]





Query: Preparing for job interviews

Top 5 most similar sentences in corpus:
tensor(28)
Product Manager Interview Preparation (Score: 0.5231)
tensor(21)
Front-End Interview Prep (Score: 0.4897)
tensor(29)
VR Interview Prep (Score: 0.4790)
tensor(23)
Android Interview Prep (Score: 0.4760)
tensor(22)
Machine Learning Interview Preparation (Score: 0.4552)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]





Query: Make a web application from scratch

Top 5 most similar sentences in corpus:
tensor(84)
How to Make an iOS App (Score: 0.4641)
tensor(167)
Offline Web Applications (Score: 0.4351)
tensor(64)
How to Build a Startup (Score: 0.4191)
tensor(135)
Developing Android Apps (Score: 0.3923)
tensor(190)
Build Native Mobile Apps with Flutter (Score: 0.3905)
